## Population Density

### Joining population data from census blocks to a hex grid via areal interpolation

Areal interpolation is the process of transfering data from one set of areal units to another set of areal units

In this case, we are transfering population data from census blocks $b$ to a hex cell $h$ based on the ratio of the area of the block that intersects the hex cell $A_{b \cap h}$ by the total area of the block $A_b$

### $P_h = \sum P_{b} {w_{b,h}}$

### $P_h = \sum P_{b} \frac{A_{b \cap h}}{A_b}$


With postgreSQL and PostGIS ..

```sql
-- creating a spatial index for the hex grid to speed up the intersection

DROP INDEX IF EXISTS hex_grid_200m_gix;
CREATE INDEX hex_grid_200m_gix ON hex_grid_200m USING GIST (geom);


-- creating an area column $A_b$ for the block data

ALTER TABLE zones_db ADD COLUMN area double precision;
UPDATE zones_db SET area = ST_AREA(geom);

-- Then we join the block populationo data to the spatial boundaries, $P_b$

CREATE TABLE temp_DB_pop AS ( 
    SELECT
    zones_DB.geom AS geom,
    zones_DB.dbuid AS dbuid,
    zones_DB.area AS area,
    table_DB_pop_2016.pop2016int AS pop2016int
    FROM zones_DB
    INNER JOIN table_DB_pop_2016 ON table_DB_pop_2016.dbuid = zones_DB.dbuid
);    

-- create a spatial index for this block polygon file

DROP INDEX IF EXISTS temp_DB_pop_gix;
CREATE INDEX temp_DB_pop_gix ON temp_DB_pop USING GIST (geom); 

-- intersecting with the hex grid, then grouping to get the a set of weights of block to hex grid

CREATE TABLE temp_int_DB_hex AS
(
SELECT
hex_grid_200m.id AS hexid,
temp_DB_pop.dbuid AS dbuid,
temp_DB_pop.area AS area_full,
ST_Intersection(hex_grid_200m.geom,temp_DB_pop.geom) AS geom
FROM
hex_grid_200m INNER JOIN temp_DB_pop ON ST_Intersects(hex_grid_200m.geom,temp_DB_pop.geom)
);

-- update area of intersected geoms

ALTER TABLE temp_int_DB_hex ADD COLUMN area_int double precision;
UPDATE temp_int_DB_hex SET area_int = ST_AREA(geom);

ALTER TABLE temp_int_DB_hex ADD COLUMN area_ratio double precision;
UPDATE temp_int_DB_hex SET area_ratio = area_int / area_full;

-- grouping by hex and DB unique IDs - i.e. this is a weights table that can be used for apportioning data

CREATE TABLE weights_db_hex AS
(
SELECT
hexid,
dbuid,
sum(area_ratio) AS weight
FROM
temp_int_DB_hex
GROUP BY dbuid, hexid
ORDER BY dbuid, hexid
);

-- we can delete the temp tables if we want, but they take a little while to compute so it may be nice to keep them if space isnt an issue

DROP TABLE temp_DB_pop;
DROP TABLE temp_int_DB_hex;

```

We now use the weights table to apportion the population data to the hex grid

```sql
-- joining the population to the weights table
-- multiplying the pop by the weight and aggregating by hex grid - also normalize as persons/km2

DROP TABLE IF EXISTS out_data_hex_pop2016;
CREATE TABLE out_data_hex_pop2016 AS (
    WITH temp_weight_join AS (
        SELECT
        weights_db_hex.dbuid AS dbuid,
        weights_db_hex.hexid AS hexid,
        weights_db_hex.weight AS weight,
        table_DB_pop_2016.pop2016int AS pop2016int
        FROM weights_db_hex
        INNER JOIN table_DB_pop_2016 ON table_DB_pop_2016.dbuid = weights_db_hex.dbuid
    ) 
    SELECT
    temp_weight_join.hexid AS hexid,
    SUM(temp_weight_join.weight * temp_weight_join.pop2016int) AS pop2016
    FROM temp_weight_join GROUP BY temp_weight_join.hexid
);
  
-- Create a pop density column                                                                 
                                                                  
ALTER TABLE out_data_hex_pop2016 ADD COLUMN popdensity2016 double precision;
UPDATE out_data_hex_pop2016 SET popdensity2016 = pop2016 / (34641.0161513719 / (1000 * 1000));
                                                                  
-- lets write that to a file

\COPY out_data_hex_pop2016 TO 'output_hex_data/out_data_hex_pop2016.csv' WITH (FORMAT CSV, HEADER);
                                                                  
```